# This is used for running data vs human_msigdb_c3 annotations located in: 
### /nvme/bins/iPAGEv1.0/PAGE_DATA/ANNOTATIONS/human_ensembl_msigdb_c3

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import scipy.stats


### get index of file names to loop through 

In [2]:
%%bash

cd fc_values
ls FC* > filelist.txt
cd ..

### get ensembl genes represented in iPAGE database

In [3]:
%%bash 

cp /nvme/bins/iPAGEv1.0/PAGE_DATA/ANNOTATIONS/human_ensembl_msigdb_c3/human_ensembl_msigdb_c3_index.txt human_ensembl_index.txt
awk '{print $1}' human_ensembl_index.txt > ensembl_index.txt

### make sure that genes in file match the index 

In [4]:
###################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
###################REMEMBER TO SPECIFY INDEX USED IN THE IPAGE COMMAND!!!!!!!!
###################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


#####################
#######read in data from logfc calculations
#####################
####define index file to loop through
index_file = "fc_values/filelist.txt"
####define matching ensg index
GO_index_filename = "ensembl_index.txt"
genes_with_GOs = []
with open(GO_index_filename,'r') as rf:
    for line in rf:
        gene_name = line.split('\n')[0]
        genes_with_GOs.append(gene_name)
genes_with_GOs_set = set(genes_with_GOs)
####open file and loop through each line
###
iPAGE_sh = "iPAGE_JY.sh"
cwd = os.getcwd()+"/"
os.system("rm "+iPAGE_sh)
###
with open(index_file,'r') as rf:
    for line in rf:
        #####do this to each file
        ###read in file into pandas df
        FC_file = "fc_values/"+line.split('\n')[0]
        FC_df = pd.read_csv(FC_file,sep=',',index_col=0)
        FC_df.columns = ['log2FoldChange']
        FC_df.index.names = ['Gene']
        ###filter
        FC_df = FC_df.loc[[x for x in FC_df.index if x in genes_with_GOs_set]]
        ###save
        save_file = "fc_values/filt_"+line.split('\n')[0]
        FC_df.to_csv(save_file,sep='\t',header=True)
        ###write final iPAGE command to an ipage .sh file
        iPAGE_cmd = "perl /nvme/bins/iPAGEv1.0/page.pl --expfile "+cwd+save_file+" --species=human_ensembl_msigdb_c3 --exptype=continuous --ebins=9 \n"
        with open(iPAGE_sh, "a") as myfile:
            myfile.write(iPAGE_cmd)


## then run the command from this folder:

nohup sh run.sh iPAGE_JY.sh &

In [5]:
%%bash

nohup sh run.sh iPAGE_JY.sh &

perl /nvme/bins/iPAGEv1.0/page.pl --expfile /avicenna/Johnny/scRNAseq/metastatic_evolution/ipage_human_ensembl_msigdb_c3/fc_values/filt_FC_LVM1_1.csv --species=human_ensembl_msigdb_c3 --exptype=continuous --ebins=9
done
perl /nvme/bins/iPAGEv1.0/page.pl --expfile /avicenna/Johnny/scRNAseq/metastatic_evolution/ipage_human_ensembl_msigdb_c3/fc_values/filt_FC_LVM1_2.csv --species=human_ensembl_msigdb_c3 --exptype=continuous --ebins=9
homologiesfile: 
genelist: 
REMOVING DUPLICATES
perl /nvme/bins/iPAGEv1.0/SCRIPTS/remove_homologous_sequences_withseed.pl -expfile /avicenna/Johnny/scRNAseq/metastatic_evolution/ipage_human_ensembl_msigdb_c3/fc_values/filt_FC_LVM1_1.csv -quantized 0 -genelist /nvme/bins/iPAGEv1.0/PAGE_DATA/ANNOTATIONS/seqnames/human_ensembl_msigdb_c3.txt -dupfile /nvme/bins/iPAGEv1.0/PAGE_DATA/ANNOTATIONS/human_ensembl_msigdb_c3/human_ensembl_msigdb_c3.homologies -outfile /avicenna/Johnny/scRNAseq/metastatic_evolution/ipage_human_ensembl_msigdb_c3/fc_values/filt_FC_LVM1_1.csv